<a href="https://colab.research.google.com/github/prateek0489/100daysToDS/blob/master/FirstNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai

In [0]:
# import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import string
import io
import os
pd.set_option('display.max_colwidth', 100)


In [35]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
stopwords = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()

In [0]:
training_data=pd.read_csv("train.csv")

In [0]:
re.fullmatch("((https?|ftp|smtp|http?):\/\/)",training_data.tweet[1]) 

In [0]:
^((https?|ftp|smtp):\/\/)?(www.)?[a-z0-9]+\.[a-z]+(\/[a-zA-Z0-9#]+\/?)*$

In [52]:
string1="Hello"
re.findall('[A-Z a-z+]',string1)

['H', 'e', 'l', 'l', 'o']

In [37]:
training_data.tweet[1]

'Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/'

In [25]:
def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in stopwords]
    return text

training_data['body_text_nostop'] = training_data['tweet'].apply(lambda x: clean_text(x.lower()))

training_data.head()

,id,label,tweet,body_text_nostop
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,"[fingerprint, pregnancy, test, httpsgooglh1mfq..."
1,2,0,Finally a transparant silicon case ^^ Thanks t...,"[finally, transparant, silicon, case, thanks, ..."
2,3,0,We love this! Would you go? #talk #makememorie...,"[love, would, go, talk, makememories, unplug, ..."
3,4,0,I'm wired I know I'm George I was made that wa...,"[im, wired, know, im, george, made, way, iphon..."
4,5,1,What amazing service! Apple won't even talk to...,"[amazing, service, apple, wont, even, talk, qu..."


In [28]:
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()
def lemmatizing(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

training_data['body_text_lemmatized'] = training_data['body_text_nostop'].apply(lambda x: lemmatizing(x))

training_data.head(10)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,id,label,tweet,body_text_nostop,body_text_lemmatized
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...,"[fingerprint, pregnancy, test, httpsgooglh1mfq...","[fingerprint, pregnancy, test, httpsgooglh1mfq..."
1,2,0,Finally a transparant silicon case ^^ Thanks t...,"[finally, transparant, silicon, case, thanks, ...","[finally, transparant, silicon, case, thanks, ..."
2,3,0,We love this! Would you go? #talk #makememorie...,"[love, would, go, talk, makememories, unplug, ...","[love, would, go, talk, makememories, unplug, ..."
3,4,0,I'm wired I know I'm George I was made that wa...,"[im, wired, know, im, george, made, way, iphon...","[im, wired, know, im, george, made, way, iphon..."
4,5,1,What amazing service! Apple won't even talk to...,"[amazing, service, apple, wont, even, talk, qu...","[amazing, service, apple, wont, even, talk, qu..."
5,6,1,iPhone software update fucked up my phone big ...,"[iphone, software, update, fucked, phone, big,...","[iphone, software, update, fucked, phone, big,..."
6,7,0,Happy for us .. #instapic #instadaily #us #son...,"[happy, us, instapic, instadaily, us, sony, xp...","[happy, u, instapic, instadaily, u, sony, xper..."
7,8,0,New Type C charger cable #UK http://www.ebay.c...,"[new, type, c, charger, cable, uk, httpwwwebay...","[new, type, c, charger, cable, uk, httpwwwebay..."
8,9,0,Bout to go shopping again listening to music #...,"[bout, go, shopping, listening, music, iphone,...","[bout, go, shopping, listening, music, iphone,..."
9,10,0,Photo: #fun #selfie #pool #water #sony #camera...,"[photo, fun, selfie, pool, water, sony, camera...","[photo, fun, selfie, pool, water, sony, camera..."
